In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from researchlib.single_import import *

Using GPU 0: GeForce RTX 2080 Ti
Driver: 418.56


In [3]:
import torchvision
x = np.load('meow128.npy')
x = (x * 2) - 1
train_loader = FromNumpy((x, x), 1, 32)
# train_loader = VisionDataset(vision.CIFAR10, 256, True)

In [4]:
runner = Runner(GANModel(AutoGAN_G(128, 32), AutoGAN_D(128, 32), latent_vector_len=512),#, condition_vector_len=[10, 10], condition_onehot=True),
                train_loader, 
                None, 
                'adam_gan',
                GANLoss('relative-averaged-lsgan'),
                monitor_state='g_loss',
                monitor_mode='min',
                larc=False)

In [5]:
runner.init_model('default')

In [ ]:
runner.fit(150, (5e-5, 2e-4), 'fixed')

Output()

In [ ]:
# img = runner.model.sample(100, condition_data=list(range(10))*10)
runner.model.cpu()
img = runner.model.sample(100)
img = (img+1)/2
import matplotlib.pyplot as plt
fig, arr = plt.subplots(10, 10, figsize=(20, 20))
for i in range(100):
    r = img[i].detach().cpu().numpy().transpose(1,2,0)
    arr[i//10][i%10].axis('off')
    arr[i//10][i%10].imshow(r)#, cmap='gray')
plt.tight_layout()
plt.show()

In [ ]:
runner.report()

In [ ]:
for i in runner.model.generator.main.children():
    try:
        print('@@', torch.sigmoid(i.gamma))
    except:
        pass

In [ ]:
torch.sigmoid(0.3)